In [1]:
import json
import torch
import transformers as tf
import numpy as np

from sklearn.linear_model import LogisticRegression

In [2]:
with open('./data/train.jsonl', 'r') as js_file:
    js_list = list(js_file)
line=js_list[0]
sample=json.loads(line)
sample

{'word': 'place',
 'sentence1': 'Do you want to come over to my place later?',
 'sentence2': 'A political system with no place for the less prominent groups.',
 'idx': 0,
 'label': False,
 'start1': 31,
 'start2': 27,
 'end1': 36,
 'end2': 32,
 'version': 1.1}

In [3]:
'''
Load sentence in to numpy. The symbol '+' here denotes seperation
'''
sen1=np.array([json.loads(i)["sentence1"] for i in js_list])
sen2=np.array([json.loads(i)["sentence2"] for i in js_list])
sep=np.array([' + ']*len(sen1))# sep array
sen_full=np.char.add(sen1,sep)
sen_full=np.char.add(sen_full,sen2)

idx=np.array([json.loads(i)["idx"] for i in js_list]) #load index

In [8]:
#Load pre trained BERT in the package transformer
model_class, tokenizer_class, pretrained_weights=(tf.BertModel, tf.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [10]:
'''
Tokenize two sentences with int
'''
tokenized_sen=np.array([tokenizer.encode(sen1[i],sen2[i],add_special_tokens=True) for i in range(5000)]);

<ipython-input-10-8d07cd27aadc>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tokenized_sen=np.array([tokenizer.encode(sen1[i],sen2[i],add_special_tokens=True) for i in range(5000)]);


In [11]:
'''
Padding
Align the length to the max length
'''
def find_max(npa):
    max_len = 0
    for i in npa:
        if len(i) > max_len:
            max_len = len(i)
    return(max_len)
def padding(npa):
    max_len=find_max(npa)
    return (np.array([i + [0]*(max_len-len(i)) for i in npa]))

In [12]:
token_sen_afp=padding(tokenized_sen)
mask = np.where(token_sen_afp != 0, 1, 0)#1 is non padding part 0 o.w
inp = torch.tensor(token_sen_afp).to(torch.int64)#convert to tensor and in64 for the BERT transformer
mask = torch.tensor(mask)

In [14]:
'''
Error said require about 4GB RAM
my computer has about 16GB RAM and at least 7GB free when I ran it.
I also tried CUDA but will give error
'''

'''
with torch.no_grad():
    outcome = model(inp,mask)
    
features = outcome[0][:,0,:].numpy()

lgR = LogisticRegression(max_iter=1000)
lgR.fit(features, idx)
'''

'\nwith torch.no_grad():\n    outcome = model(inp,mask)\n'